In [1]:
import pandas as pd
import numpy as np

import torch 
import torch.nn.functional as F

In [2]:
if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    torch.cuda.set_device(current_device)
    device = torch.device('cuda:{}'.format(current_device))
    print('Using GPU: {}'.format(torch.cuda.get_device_name(current_device)))
else:
    device = torch.device('cpu')

Using GPU: GeForce MX250


In [3]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

In [4]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x_test = test_df.values/255

In [7]:
y = train_df.label.values

In [8]:
x = train_df.iloc[:,1:].values/255

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.2) 

In [11]:
trainTorch_x = torch.from_numpy(train_x).type(torch.FloatTensor)
trainTorch_y = torch.from_numpy(train_y).type(torch.LongTensor)


valTorch_x = torch.from_numpy(val_x).type(torch.FloatTensor)
valTorch_y = torch.from_numpy(val_y).type(torch.LongTensor) 

testTorch_x = torch.from_numpy(np.array(x_test)).type(torch.FloatTensor)

In [12]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim

In [13]:
train = torch.utils.data.TensorDataset(trainTorch_x, trainTorch_y)
val = torch.utils.data.TensorDataset(valTorch_x, valTorch_y)
test = torch.utils.data.TensorDataset(testTorch_x)

In [14]:
train_loader = DataLoader(train, batch_size = 100, shuffle = False)
val_loader = DataLoader(val, batch_size = 100, shuffle = False)
test_loader = DataLoader(test, batch_size = 100, shuffle = False)

In [15]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = torch.nn.Linear(784,512)
        self.l2 = torch.nn.Linear(512,256)
        self.l3 = torch.nn.Linear(256,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,10)
        
    def forward(self,x):
        x = x.view(-1,784)
#         reshape
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
#         注意最后一层不激活

In [16]:
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01,momentum=0.5)

In [17]:
# 将单轮循环封装为一个函数

def train_func(epoch):
    running_loss = 0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,target = data
        optimizer.zero_grad()
        
        
#         forward and backward and update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d,%5d] loss: %.3f' % (epoch + 1,batch_idx + 1,running_loss / 300))
            running_loss =0.0
            

In [18]:
def test_func():
    correct = 0
    total = 0
    with torch.no_grad():
#         执行之后在下面代码就不会执行梯度
        for data in val_loader:
            images,labels = data
#             拿数据
            outputs = model(images)
#             做预测，拿到的结果是一个矩阵，每一行都是一个独热向量
            _, predicted = torch.max(outputs.data,dim = 1)
#           返回 最大值 和 每一行的最大值下标
#           指定沿着维度1（往下 行是第0个维度，向右 列是第一个维度）
            total += labels.size(0)
#             label是一个N 1元组 size 取 0 就是？

            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [19]:
def predict_func():
    with torch.no_grad():
        outputs = model(testTorch_x)
        index, predicted = torch.max(outputs.data,dim = 1)
    ans2 = predicted
    submission_file2 = pd.read_csv('./sample_submission.csv')
    submission_file2.Label = ans2
    submission_file2.to_csv('submission_nn.csv', index=False)